In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import re

driver = webdriver.ChromiumEdge()
# driver = webdriver.Chrome(service=Service("C:/chromedriver.exe"))
driver.maximize_window()
action = ActionChains(driver=driver)


In [3]:
def find_element_by_xpath(xpath, data_type):
    try:
        if data_type == "text":
            obj = driver.find_element(By.XPATH, xpath).text
            return obj
        elif data_type == "src" or data_type == "href" or data_type == "innerHTML":
            obj = driver.find_element(By.XPATH, xpath).get_attribute(data_type)
            return obj
        else:
            obj = driver.find_element(By.XPATH, xpath)
            return obj
    except:
        return None

def find_elements_by_xpath(xpath):
    try:
        obj = driver.find_elements(By.XPATH, xpath)
        return obj
    except:
        return None

def go_to_bottom(going_value):
    body = driver.find_element(By.TAG_NAME, "body")
    driver.implicitly_wait(5)
    for _ in range(going_value):
        body.send_keys(Keys.PAGE_DOWN)

def close_driver():
    driver.close()

In [4]:
def target_required_page(target_url):
    driver.get(target_url)
    driver.implicitly_wait(5)
    go_to_bottom(10)
    driver.execute_script("document.body.style.zoom='25%'")
    time.sleep(2)

In [5]:
def data_section(soup, listing_id, state, city):
    data_dict = {}
    data_dict['group_id'] = listing_id
    data_dict['user_id'] = ""
    data_dict['state'] = state
    data_dict['usa_dropdown'] = state
    data_dict['city'] = city
    data_dict['city_dropdown'] = city
    data_dict['group_name'] = find_element_by_xpath("//h1", "text")
    data_dict['post_location'] = find_element_by_xpath('//div[@data-stid="content-hotel-address"]', "text")
    try:
        data_dict['property_beds'] = int(re.sub(r"[A-Za-z\s+]", "", find_element_by_xpath('//div[@class="uitk-layout-flex uitk-layout-flex-flex-wrap-wrap"]/span[contains(text(), "bedroom")]', "text")))
    except:
        data_dict['property_beds'] = None
    try:
        data_dict['property_baths'] = int(re.sub(r"[A-Za-z\s+]", "", find_element_by_xpath('//div[@class="uitk-layout-flex uitk-layout-flex-flex-wrap-wrap"]/span[contains(text(), "bathroom")]', "text")))
    except:
        data_dict['property_baths'] = None
    try:
        data_dict['sleeps'] = int(re.sub(r"[A-Za-z\s+]", "", find_element_by_xpath('//div[@class="uitk-layout-flex uitk-layout-flex-flex-wrap-wrap"]/span[contains(text(), "Sleep")]', "text")))
    except:
        data_dict['sleeps'] = None
    try:
        data_dict['size'] = int(re.sub(r"[A-Za-z\s+]", "", find_element_by_xpath('//div[@class="uitk-layout-flex uitk-layout-flex-flex-wrap-wrap"]/span[contains(text(), "sq ft")]', "text")))
    except:
        data_dict['size'] = None
    data_dict['price_per_night'] = find_element_by_xpath('//div[div[text() = "per night"]]/div/span/div', "text")
    data_dict['group_description'] = find_element_by_xpath('//div[@data-stid="content-item"]//div[@data-stid="content-markup"]', "innerHTML")
    amen_list = ["Essentials", "Kitchen", "Pool & spa", "Outside", "Entertainment", "Baby & toddler", "Laundry", "Parking", "Safety", "Location type", "Nearby activities", "Suitability", "Accessibility", "Breakfast", "Services"]
    try:
        click_on_amen = find_element_by_xpath('//button[contains(text(), "See all ") and contains(text(), "amenities")]', 'none')
        driver.execute_script('arguments[0].click();', click_on_amen)
    except:
        pass
    try:
        all_amen = re.sub(r"[\n]", ", ", find_element_by_xpath('//section[@class="uitk-centered-sheet uitk-sheet uitk-centered-sheet-medium"]//div[@class="uitk-card-content-section"]', "text"))
        for a in amen_list:
            if a in all_amen:
                all_amen = all_amen.replace(a+", ", "")
        all_amen = all_amen.split(", ")
        all_amen = set(all_amen)
        all_amen = ",".join(all_amen)
        all_amen = all_amen.lower().replace(" ", "_")
        all_amen = all_amen.replace(",", ", ") 
        data_dict['ammenities_dropdown'] = all_amen
    except:
        data_dict['ammenities_dropdown'] = None
    for a in amen_list:
        try:
            data_dict[f'{a}'] = re.sub(r"[\n]", ", ", find_element_by_xpath(f'//div[div/div/h3[text() = "{a}"]]/div[2]', 'text'))
        except:
            data_dict[f'{a}'] = None
    try:
        cross = find_element_by_xpath('//button[@class="uitk-layout-flex-item uitk-layout-flex-item-flex-shrink-0 uitk-toolbar-button-v2 uitk-toolbar-button-v2-icon-only"]', 'none')
        driver.execute_script("arguments[0].click();", cross)
    except:
        pass
    line = find_element_by_xpath('//div[@class="uitk-text uitk-type-300 uitk-text-default-theme" and  contains(text(), "maximum attendees: ")]', 'text')
    if line is not None:
        l1 = line[line.index("; "):].replace("; ", '')
        data_dict['max_attendees'] = l1.replace('maximum attendees:', '')
    else:
        data_dict['max_attendees'] = line
    try:
        data_dict['total_reviews'] = int(re.sub(r"[A-Za-z+]", "", find_element_by_xpath('//button[@data-stid="reviews-link"]',"text")))
    except:
        data_dict['total_reviews'] = None
    data_dict['house_rules'] = find_element_by_xpath('//div[@id="House Rules"]', 'innerHTML')
    data_dict['important_information'] = find_element_by_xpath('//section[@id="ImportantInfo"]', 'innerHTML')
    data_dict['about_neighbourhood'] = find_element_by_xpath('//div[h2[text() = "About the neighborhood"]]//section', "innerHTML")
    data_dict['whats_nearby'] = find_element_by_xpath("""//div[@class="uitk-layout-flex-item" and h4[text() = "What's nearby"]]""", "innerHTML")
    data_dict["restaurants"] = find_element_by_xpath('//div[@class="uitk-layout-flex-item" and h4[text() = "Restaurants"]]', "innerHTML")
    data_dict['getting_around'] = find_element_by_xpath('//div[@class="uitk-layout-flex-item" and h4[text() = "Getting around"]]', 'innerHTML')
    go_to_image = find_element_by_xpath('//img[@alt="Vrbo logo"]', 'none')
    action.scroll_to_element(go_to_image)
    time.sleep(1)
    open_all = find_element_by_xpath('//button[@class="uitk-image-link"]', 'none')
    action.click(open_all).perform()
    get_img = find_elements_by_xpath('//figure[@class="uitk-image uitk-image-ratio-16-9 uitk-image-ratio"]/div/img')
    for i in range(0, 300, 3):
        try:
            action.scroll_to_element(get_img[i]).perform()
            time.sleep(0.3)
        except:
            break
    action.scroll_to_element(get_img[-1]).perform()
    data_dict['images'] = [i.get_attribute('src').replace("?impolicy=resizecrop&rw=1200&ra=fit", "") for i in get_img]
    cross_imgs = find_element_by_xpath('//div[@class="uitk-layout-flex uitk-layout-flex-align-items-center uitk-toolbar"]//button[@aria-expanded="false"]', "none")
    action.click(cross_imgs).perform()
    time.sleep(1)
    data_dict['current_url'] = driver.current_url.strip()
    return data_dict

In [6]:
def host_section(soup, listing_id):
    host = {}
    host['group_id'] = listing_id
    host['property_manager'] = find_element_by_xpath('//div[@data-stid="content-item" and div/div/h3[text() = "Property manager"]]//div[@class="uitk-text uitk-type-300 uitk-type-regular uitk-text-standard-theme uitk-layout-flex-item"]', "text")
    host['propert_manager_image'] = find_element_by_xpath('//div[@data-stid="content-item" and div/div/h3[text() = "Property manager"]]//div[@data-stid="content-image"]//img', 'src')
    host['language'] = find_element_by_xpath('//div[@data-stid="content-item" and div/div/h3[text() = "Languages"]]/div[2]', "text")
    host['host'] = find_element_by_xpath('//section[@id="Host"]', 'innerHTML')
    return host

In [7]:
def review_section(soup, listing_id):
    all_reviews = []
    go_to_bottom(7)
    try:
        reviews = soup.find_all("article", {"itemprop": "review"})
        for i, review in enumerate(reviews):
            rev_dict = {
                'review_id': 0,
                'group_id': listing_id,
                'review_date': review.find('span', {'itemprop': 'datePublished'}).text if review.find('span', {'itemprop': 'datePublished'}) else None,
                'review_by': review.find('h4').text if review.find('h4') else None,
                'review_title': review.find('h5', {'data-stid': 'review_section_title'}).text if review.find('h5', {'data-stid': 'review_section_title'}) else None,
                'review_body': review.find('span', {'itemprop': 'description'}).text if review.find('span', {'itemprop': 'description'}) else None,
                'review_rating': review.find('section', {'itemprop': 'reviewRating'}).text if review.find('section', {'itemprop': 'reviewRating'}) else None,
                'created_at': 0
            }

            all_reviews.append(rev_dict)
        return all_reviews
    except Exception as e:
        print(e)


In [8]:
def open_faq_first():
    scroll_to_freq = find_element_by_xpath('//h2[text() = "Frequently asked questions"]', 'none')
    if scroll_to_freq is not None:
        action.scroll_to_element(scroll_to_freq).perform()
    go_to_bottom(2)
    try:
        freq = driver.find_elements(By.XPATH, '//section[@class="uitk-expando-list"]/div/details')
        driver.implicitly_wait(3)
        for i, f in enumerate(freq):
            if i == 0:
                time.sleep(2)
                action.click(f).perform()
                time.sleep(1)
            else:
                time.sleep(1)
                action.click(f).perform()
                time.sleep(1)
    except:
        pass

In [9]:
def bed_section(soup, listing_id):
        beds = []
        baths = []
        
        try:
            click_on_link = find_element_by_xpath('//button[text() = "See all rooms and beds details"]', 'none')
            driver.execute_script('arguments[0].click();', click_on_link)
            time.sleep(1)
        except:
            pass
            
        all_beds = driver.find_elements(By.XPATH, '//section[@class="uitk-centered-sheet uitk-sheet uitk-centered-sheet-medium"]/div[3]/div[1]/div/div/div[2]/div/div[@class="uitk-layout-grid-item uitk-layout-grid-item-has-column-start"]')
        driver.implicitly_wait(3)
        if all_beds:
            for i, a in enumerate(all_beds):
                bed_dict = {
                    'bed_id': 0,
                    'group_id': listing_id,
                    'bed_type': a.find_element(By.XPATH, './/div[1]').text,
                    'bed_description': a.find_element(By.XPATH, './/div[2]/div[2]').text,
                    'created_at': 0
                }
                beds.append(bed_dict)

        all_baths = driver.find_elements(By.XPATH, '//section[@class="uitk-centered-sheet uitk-sheet uitk-centered-sheet-medium"]/div[3]/div[2]/div/div/div[2]/div/div[@class="uitk-layout-grid-item uitk-layout-grid-item-has-column-start"]')
        driver.implicitly_wait(3)
        if all_baths:
            for i, a in enumerate(all_baths):
                bath_dict = {
                    "bath_id": 0,
                    "group_id": listing_id,
                    "bath_type": a.find_element(By.XPATH, './/div[1]').text,
                    "bath_description": a.find_element(By.XPATH, './/div[2]/div[2]').text,
                    "created_at": 0
                }
                baths.append(bath_dict)
        try:
            cross = find_element_by_xpath('//section[@class="uitk-centered-sheet uitk-sheet uitk-centered-sheet-medium"]/div[2]/button', 'none')
            driver.execute_script('arguments[0].click();', cross)
            time.sleep(1)
        except:
            pass
        return beds, baths

In [10]:
def faqs_section(soup, listing_id):
    all_faq = []
    try:
        freq = find_elements_by_xpath('//section[@class="uitk-expando-list"]/div/details')
        for i, f in enumerate(freq):
            faq = {
                "Faq_id": 0,
                'group_id': listing_id,
                'Question': f.find_element(By.XPATH, './/summary').text,
                'Answer': f.find_element(By.XPATH, './/div/div/p').text,
                'created_at': 0
            }
            all_faq.append(faq)
        return all_faq
    except:
        pass

In [11]:
starting_id = 50
file_path = "C:/IMP CODES/VOCATION ALL LINK+DATA/VocationalDATA/los_angles2.xlsx"
state = "california"
city = "los_angles"
with open("C:/IMP CODES/VOCATION ALL LINK+DATA/VocationalLinks/01_Los Angeles.csv") as file:
    next(file)
    for f in file:
        target_required_page(f)
        open_faq_first()
        soup = BeautifulSoup(driver.page_source, "html.parser")
        data_dict = data_section(soup, starting_id, state, city)
        for key, value in data_dict.items():
            print(str(key)+": "+str(value))
        host = host_section(soup, starting_id)
        for h in host.items():
            print(h)
        reviews = review_section(soup, starting_id)
        for r in reviews:
            print(r)
        beds, baths = bed_section(soup, starting_id)
        for b in beds:
            print(b)
        for b in baths:
            print(b)
        faq = faqs_section(soup, starting_id)
        for f in faq:
            print(f)
        df1 = pd.DataFrame([data_dict])
        dfh = pd.DataFrame([host])
        df2 = pd.DataFrame(reviews)
        df3 = pd.DataFrame(beds)
        df4 = pd.DataFrame(baths)
        df5 = pd.DataFrame(faq)

        if os.path.exists(file_path):
            with pd.ExcelWriter(file_path, engine="openpyxl", mode='a', if_sheet_exists='overlay') as writer:
                df1.to_excel(writer, sheet_name="Data File", index=False, header=False, startrow=writer.sheets["Data File"].max_row)
                df2.to_excel(writer, sheet_name="Review File", index=False, header=False, startrow=writer.sheets["Review File"].max_row)
                df3.to_excel(writer, sheet_name="Bed File", index=False, header=False, startrow=writer.sheets["Bed File"].max_row)
                df4.to_excel(writer, sheet_name="Bath File", index=False, header=False, startrow=writer.sheets["Bath File"].max_row)
                df5.to_excel(writer, sheet_name="FAQ's File", index=False, header=False, startrow=writer.sheets["FAQ's File"].max_row)
                dfh.to_excel(writer, sheet_name="Host File", index=False, header=False, startrow=writer.sheets["Host File"].max_row)
        else:
            with pd.ExcelWriter(file_path, engine="openpyxl") as writer:
                df1.to_excel(writer, sheet_name="Data File", index=False)
                df2.to_excel(writer, sheet_name="Review File", index=False)
                df3.to_excel(writer, sheet_name="Bed File", index=False)
                df4.to_excel(writer, sheet_name="Bath File", index=False)
                df5.to_excel(writer, sheet_name="FAQ's File", index=False)
                dfh.to_excel(writer, sheet_name="Host File", index=False)

        starting_id = starting_id + 1

close_driver()





FileNotFoundError: [Errno 2] No such file or directory: 'C:/IMP CODES/VOCATION ALL LINK+DATA/VocationalLinks/01_Los Angeles.csv'